gpt를 이용한 구현 

In [4]:
import os
import sys

# 프로젝트 루트 경로 설정
project_root = os.path.dirname(os.getcwd())
sys.path.append(project_root)

# 장고 설정 모듈 설정
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'myproject.settings')

# 장고 설정 적용
import django
django.setup()

In [2]:
os.getcwd()

'c:\\proj\\git\\SERVICE FOLDER\\1eok\\myproject\\myproject'

In [3]:
from myapp.models import TrainingData
import json
from channels.db import database_sync_to_async
import asyncio

# 예시 데이터 (실제 데이터로 대체)
input_data = {'feature1': 1, 'feature2': 2}
output_data = {'label': 0}

# JSON으로 변환
input_data_json = json.dumps(input_data)
output_data_json = json.dumps(output_data)

# 데이터베이스에 저장하는 동기식 함수
def save_to_database(input_data_json, output_data_json):
    training_data = TrainingData(input_data=input_data_json, output_data=output_data_json)
    training_data.save()

# 동기식 함수를 비동기식으로 실행
@database_sync_to_async
def save_data_async(input_data_json, output_data_json):
    save_to_database(input_data_json, output_data_json)

# 이벤트 루프를 사용하여 비동기식 함수 실행
await save_data_async(input_data_json, output_data_json)

In [4]:
import asyncio
import nest_asyncio
from myapp.models import TrainingData
from channels.db import database_sync_to_async

nest_asyncio.apply()

# 모든 TrainingData 객체를 가져오는 동기식 함수
def get_all_training_data():
    all_training_data = TrainingData.objects.all()
    return all_training_data

# 동기식 함수를 비동기식으로 실행
@database_sync_to_async
def get_all_training_data_async():
    return get_all_training_data()

# 비동기 함수를 실행하고 결과를 받는 새로운 함수
async def main():
    all_training_data = await get_all_training_data_async()
    for data in all_training_data:
        await asyncio.sleep(0)
        print(f"ID: {data.id}, Input Data: {data.input_data}, Output Data: {data.output_data}, Timestamp: {data.timestamp}")

# 이벤트 루프를 사용하여 main() 함수 실행
await main()

SynchronousOnlyOperation: You cannot call this from an async context - use a thread or sync_to_async.

gpt를 이용해서 실패 

이하 빙을 이용한 구현

In [1]:
from asgiref.sync import sync_to_async
from myapp.models import Product

@sync_to_async
def create_product(name, price):
    product = Product(name=name, price=price)
    product.save()

await create_product('example2', 11000)

ModuleNotFoundError: No module named 'myapp'

In [8]:
from asgiref.sync import sync_to_async

@sync_to_async
def get_products():
    return list(Product.objects.all())

products = await get_products()

In [5]:
print(products)

[<Product: Product object (1)>]


In [9]:
for product in products:
    print(product.name, product.price)

example 1000
example2 11000


실제적으로 모델1에서 사용했던 데이터를 db에 저장해보자

1. 기존 데이터 생김새 파악

In [2]:
#시각화용 임포트 모음
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objs as go

import tensorflow as tf
import numpy as np
import pandas as pd

#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
# test_file = tf.keras.utils.get_file( 'test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
# test_text = open(test_file,'rb').read().decode(encoding='utf-8')


In [ ]:
print(train_text)

In [3]:
df_train = pd.DataFrame({
    'text':[ row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0 ],
    'sentiment': [ int(row.split('\t')[2]) for row in train_text.split('\n')[1:] if row.count('\t')>0]
})

#데이터 크기 지정
# df_train = df_train[::6]

In [4]:
df_train.loc[0][1]
#앞의 숫자는 행번호 뒤의 숫자는 0일때 context, 1일때 label
df_train.loc[3][0]


'교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'

기존 데이터 context의 길이를 파악해서 db에 구축해야하니까 길이파악

In [13]:
maxlen=0
#len(df_train)
for i in range(len(df_train)):
    if len(df_train.loc[i][0])>= maxlen:
        maxlen=len(df_train.loc[i][0])
print(maxlen)

#공백포함 158자가 최대니까 160자로 db에 저장하면 될 듯?
#1번인덱스에 저장할 내용은 0 또는 1 숫자니까 int

158


모델1에 적용한 naver 데이터 CRUD

In [19]:
from asgiref.sync import sync_to_async
from myapp.models import NaverMovieData

# 네이버 데이터 Create
@sync_to_async
def create_data_naver(content, label):
    naver = NaverMovieData(content=content, label=label)
    naver.save()

await create_data_naver('바보', 0)

In [15]:
from asgiref.sync import sync_to_async
from myapp.models import NaverMovieData

# 네이버 데이터 Read
@sync_to_async
def get_data_naver(idx): #idx번째 데이터를 가져온다.
    return NaverMovieData.objects.get(id=idx)

data_naver = await get_data_naver(1)
print(data_naver.content)
print(data_naver.label)

test111


In [18]:
from asgiref.sync import sync_to_async
from myapp.models import NaverMovieData

#네이버 데이터 Delete
@sync_to_async
def delete_data_naver(idx): #idx번째 데이터를 가져온다.
    naver = NaverMovieData.objects.get(id=idx)
    naver.delete()
    return print('{}th data delete complete'.format(idx))

await delete_data_naver(2)

2th data delete complete


In [21]:
from asgiref.sync import sync_to_async
from myapp.models import NaverMovieData

#네이버 데이터 update
@sync_to_async
def update_data_naver(idx,content, label): #idx번째 데이터를 가져온다.
    naver = NaverMovieData.objects.get(id=idx)
    naver.content = content
    naver.label = label
    naver.save()

    return print('{}th data update complete'.format(idx))

await update_data_naver(3, '바보3',1)

3th data update complete


In [ ]:
#데이터를 삭제하면 그자리가 그대로 비어있다 해결방안?
